# Importar librerias 

In [4]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import json as js
import ast as ast
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer # NLP

# Lectura de Datasets

In [5]:
# GAMES 
df_games = pd.read_json('./json/output_steam_games.json',lines=True)

# REVIEWS
rev = []
archivo1 = r'./json/australian_user_reviews.json'
with open(archivo1, encoding='utf-8') as file:
    for line in file.readlines():
        rev.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(rev)
df_reviews = df_reviews.explode('reviews')

#ITEMS
items = []
archivo2 = r'./json/australian_users_items.json'
with open(archivo2, encoding='utf-8') as file2:
    for line2 in file2.readlines():
        items.append(ast.literal_eval(line2))

df_items = pd.DataFrame(items)
df_items = df_items.explode('items')

# Limpieza de datos

# GAMES

In [7]:
#correccion de datos en precio
mask =df_games['price'].apply(lambda x: isinstance(x,str))
df_games.loc[mask,'price'] =0.00
df_games['price'] = pd.to_numeric(df_games['price'], errors='coerce')

#cambiamos tipo de datos 
df_games['early_access'] = df_games['early_access'].fillna(0).astype(int) # borramos valores nulos para cambiar dato
df_games['early_access'] = df_games['early_access'].astype(int)

#corregimos datos dentro de la columna "release_date"
def conver_fecha(fecha):
    # Lista de formatos de fecha comunes para intentar
    for fmt in ['%d %Y', '%b %Y', '%Y-%m-%d', '%Y']:
        try:
            # Intentar convertir la fecha según el formato actual
            return pd.to_datetime(fecha, format=fmt)
        except ValueError:
            # Si el formato no coincide, continuar con el siguiente
            continue

    # Si ninguno de los formatos funciona, usar regex para buscar un año de 4 dígitos
    match = re.search(r'\b(\d{4})\b', fecha)
    if match:
        # Si se encuentra un año, convertirlo a tipo datetime usando el formato '%Y'
        return pd.to_datetime(match.group(1), format='%Y')

    # Si no se puede convertir la fecha, devolver NaT (Not a Time)
    return pd.NaT

# Aplicar la función a la columna 'release_date' del DataFrame
#df_games['release_date'] = df_games['release_date'].apply(conver_fecha)


# REVIEWS 

In [2]:
df_reviews_flat = pd.json_normalize(df_reviews['reviews']) # desanida la columna reviews 

df_reviews = pd.concat([df_reviews.drop(columns=['reviews']).reset_index(drop=True), df_reviews_flat], axis=1)
#                     elimina la columna original / reseta el indice / une los datasets

df_reviews = df_reviews.replace('', None) # Reemplaza los valores vacios 


df_reviews = df_reviews.drop_duplicates() # borramos datos duplicados
f_reviews = df_reviews.dropna()#eliminamos nulos 


#cambiamos tipo de datos
# df_reviews['posted'] = pd.to_datetime(df_reviews['posted'], errors='coerce')
df_reviews['funny'] = pd.to_numeric(df_reviews['funny'], errors='coerce').fillna(0).astype(int)
df_reviews['recommend'] = pd.to_numeric(df_reviews['recommend'], errors='coerce').fillna(0).astype(int)

NameError: name 'pd' is not defined

In [1]:
df_items.head()

NameError: name 'df_items' is not defined

__*Analisis de sentimiento NLP*__

In [6]:
analyzer = SentimentIntensityAnalyzer()

def sentiment_score(review):
    sentimento = analyzer.polarity_scores(review) # Calcula los puntajes de sentimiento
    score = sentimento['compound']  # determinar si la reseña es negativa, neutral o positiva.
    if score <= -0.05:
        return 0  # Negativa
    elif score >= 0.05:
        return 2  # Positivo
    else:
        return 1  # Neutral
    
# Aplicamos la funcion
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(lambda x: sentiment_score(x) if pd.notnull(x) else 1)

df_reviews = df_reviews.drop(columns=['review'])


__*lambda x: sentiment_score(x) if pd.notnull(x) else 1:*__ asegura que solo se aplique 'sentiment_score' a reseñas que no son nulas <br>

__*(pd.notnull(x)).*__ Si la reseña es nula, se asigna directamente el valor 1 (neutral) <br>

El resultado de esta operación se almacena en una nueva columna llamada sentiment_analysis, que contendrá los valores de sentimiento (0, 1 o 2) para cada reseña.

# ITEMS

In [6]:
df_items_flat = pd.json_normalize(df_items['items']) # desanida la columna reviews



df_items = pd.concat([df_items.drop(columns=['items']).reset_index(drop=True), df_items_flat], axis=1)
#                    elimina la columna original / reseta el indice / une los datasets

df_items = df_items.replace('', None) # Reemplaza los valores vacios a none


# Guardamos como nuevo dataset 

In [7]:
df_games.to_json('./data_clean/cleaned_output_steam_games.json', orient='records', lines=True)
df_reviews.to_json('./data_clean/cleaned_user_reviews.json', orient='records', lines=True)
df_items.to_json('./data_clean/cleaned_australian_users_items.json', orient='records', lines=True)